In [2]:
import mysql.connector
from mysql.connector import Error
import datetime # Import for date objects

def connect_to_mysql():
    conn = None
    try:
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="Lemon123", # <--- IMPORTANT: Replace with your actual password
            database="LittleLemonDB"
        )
        if conn.is_connected():
            print("Successfully connected to the Little Lemon database!")
            db_info = conn.get_server_info()
            print("MySQL Server version:", db_info)
            cursor = conn.cursor()
            cursor.execute("SELECT DATABASE();")
            record = cursor.fetchone()
            print("Connected to database:", record[0])
            cursor.close()
        return conn
    except Error as e:
        print(f"Error connecting to MySQL database: {e}")
        if e.errno == Error.ER_ACCESS_DENIED_ERROR:
            print("Something is wrong with your user name or password.")
        elif e.errno == Error.ER_BAD_DB_ERROR:
            print("Database does not exist.")
        else:
            print("Please ensure your MySQL server is running and connection details are correct.")
        return None

def main():
    connection = connect_to_mysql()
    if connection:
        try:
            # --- Call GetMaxQuantity procedure ---
            print("\n--- Calling GetMaxQuantity procedure ---")
            cursor = connection.cursor()
            cursor.callproc('GetMaxQuantity')
            for result in cursor.stored_results():
                max_quantity_data = result.fetchone()
                if max_quantity_data:
                    print(f"Maximum Quantity from Bookings: {max_quantity_data[0]}")
                else:
                    print("No quantity data found in Bookings table.")
            cursor.close() # Close cursor after use

            # --- Call AddBooking procedure ---
            print("\n--- Attempting to add a new booking using AddBooking procedure ---")
            cursor = connection.cursor()
            new_booking_data = (
                "LLNEW009",  # p_BookingID (VARCHAR) - CHANGE THIS EACH TIME FOR NEW BOOKING (e.g., "LLNEW005", "LLNEW006", "LLNEW007")
                "72-055-7985", # p_CustomerID (VARCHAR) - Must exist in Customers table
                datetime.date(2025, 6, 23), # p_OrderDate (DATE)
                datetime.date(2025, 6, 25), # p_DeliveryDate (DATE)
                50.00,        # p_Cost (DECIMAL)
                75.00,        # p_Sales (DECIMAL)
                2,            # p_Quantity (INT)
                5.00,         # p_Discount (DECIMAL)
                10.00,        # p_DeliveryCost (DECIMAL)
                "Pasta",      # p_CourseName (VARCHAR)
                "Italian",    # p_CuisineName (VARCHAR)
                "Garlic Bread", # p_StarterName (VARCHAR)
                "Tiramisu",   # p_DesertName (VARCHAR)
                "Water",      # p_Drink (VARCHAR)
                "Salad"       # p_Sides (VARCHAR)
            )
            cursor.callproc('AddBooking', args=new_booking_data)
            connection.commit()
            print("AddBooking procedure called and committed.")
            for result in cursor.stored_results():
                message = result.fetchone()
                if message:
                    print(f"Procedure Message: {message[0]}")
            cursor.close() # Close cursor after use

            # --- Call UpdateBooking procedure ---
            print("\n--- Attempting to update an existing booking using UpdateBooking procedure ---")
            cursor = connection.cursor()
            booking_to_update_data = (
                "54-366-6861", # p_BookingID: Must exist in Bookings table (original data)
                datetime.date(2025, 7, 1), # p_NewOrderDate
                datetime.date(2025, 7, 3), # p_NewDeliveryDate
                5              # p_NewQuantity
            )
            cursor.callproc('UpdateBooking', args=booking_to_update_data)
            connection.commit()
            print(f"UpdateBooking procedure called and committed for Booking ID: {booking_to_update_data[0]}.")
            for result in cursor.stored_results():
                message = result.fetchone()
                if message:
                    print(f"Procedure Message: {message[0]}")
            cursor.close() # Close cursor after use

            # --- Call CancelBooking procedure ---
            print("\n--- Attempting to cancel a booking using CancelBooking procedure ---")
            cursor = connection.cursor()
            booking_to_cancel_id = "LLNEW004" # IMPORTANT: Make this match the ID you just added with AddBooking

            cursor.callproc('CancelBooking', args=(booking_to_cancel_id,)) # Note the comma for single-element tuple
            connection.commit()
            print(f"CancelBooking procedure called and committed for Booking ID: {booking_to_cancel_id}.")
            for result in cursor.stored_results():
                message = result.fetchone()
                if message:
                    print(f"Procedure Message: {message[0]}")
            cursor.close() # Close cursor after use


        except Error as e:
            print(f"Error during database operations: {e}")
            if connection.is_connected():
                connection.rollback()
                print("Transaction rolled back due to error.")
        finally:
            if connection and connection.is_connected():
                connection.close()
                print("MySQL connection is closed.")
    else:
        print("Failed to establish database connection. Cannot run main operations.")

if __name__ == "__main__":
    main()

Successfully connected to the Little Lemon database!
MySQL Server version: 8.0.42
Connected to database: littlelemondb

--- Calling GetMaxQuantity procedure ---
Maximum Quantity from Bookings: 5

--- Attempting to add a new booking using AddBooking procedure ---
AddBooking procedure called and committed.
Procedure Message: Booking added successfully.

--- Attempting to update an existing booking using UpdateBooking procedure ---
UpdateBooking procedure called and committed for Booking ID: 54-366-6861.
Procedure Message: Booking 54-366-6861 updated successfully.

--- Attempting to cancel a booking using CancelBooking procedure ---
CancelBooking procedure called and committed for Booking ID: LLNEW004.
Procedure Message: Booking LLNEW004 cancelled successfully.
MySQL connection is closed.
